In [1]:
############
# Imports #
############

import json
import os

import torch
import wandb
from torch.utils.data import DataLoader

from model.cnn_classifier import model_4D
from model.finetune import FinetuneLoop
from model.pseudomask import Pseudomasks
from model.train import ClassifierTrainLoop
from utils.data_modules import ImageDataset, TestSet, filter_dataset

##############################
# define train/test function #
##############################


run = wandb.init(
        project="VGG_CAMs",
        tags=['PseudomaskGridsearch']
)

cam_threshold_factor = None
std_from_mean = None

# define model
model_path = "nadjaflechner/VGG_CAMs/classification_model:v61"
api = wandb.Api()
artifact = api.artifact(model_path, type='model')
artifact_dir = artifact.download()
state_dict = torch.load(f"{artifact_dir}/model.pth", map_location=torch.device('cpu'))

# define dataset
GT_dir = "/Volumes/USB/Ground_truth/All_GTs"
dataset = TestSet("hs", GT_dir, normalize = False)
test_loader = DataLoader(dataset, batch_size=1, shuffle=True)

# Make pseudomask generator
pseudomask_generator = Pseudomasks(test_loader, cam_threshold_factor, overlap_threshold=0,
                                    snic_seeds=0, snic_compactness=0, finetuned=False, std_from_mean=std_from_mean)
pseudomask_generator.model_from_dict(state_dict)
pseudomask_generator.test_loop_otsu_CAMs(test_loader)



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nadja-flechner (nadjaflechner). Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  
